In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from sqlalchemy import *
from sqlalchemy_utils import database_exists, create_database
import psycopg2

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 5.0)
plt.rcParams['figure.facecolor'] = 'white'

19,307,026 submissions

5,326,298 contest submissions

In [2]:
dbname = 'codeforces'
username = 'Joy'
con = psycopg2.connect(database = dbname, user = username)
cur = con.cursor()

In [5]:
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print cur.fetchall()

[('handles',), ('problem_info',), ('submissions',), ('contests',), ('hacks',), ('user_rating',), ('problem_rating',), ('tags',)]


In [20]:
cur.execute("select * from handles")
handles = [c[0] for c in cur.fetchall()]

In [95]:
query = "select * from submissions limit 10"
pd.read_sql_query(query, con)

,handle,id,language,memoryBytes,participanttype,passedtestcount,points,relativetimeseconds,starttimeseconds,timemilliseconds,testset,verdict,contestid,problemid,problemrating
0,tourist,27036616,GNU C++11,19763200,CONTESTANT,33,3500,7077,1494523977,1294,TESTS,OK,799,G,None
1,tourist,27035049,GNU C++11,19763200,CONTESTANT,14,3500,6853,1494523753,2808,PRETESTS,SKIPPED,799,G,None
2,tourist,27030563,GNU C++11,30208000,CONTESTANT,47,3250,4891,1494521791,483,TESTS,OK,799,F,None
3,tourist,27029223,GNU C++11,13004800,CONTESTANT,2,3250,4346,1494521246,0,PRETESTS,WRONG_ANSWER,799,F,None
4,tourist,27023350,GNU C++11,13107200,CONTESTANT,16,2500,2511,1494519411,78,TESTS,WRONG_ANSWER,799,E,None
5,tourist,27022631,GNU C++11,4915200,CONTESTANT,103,2000,2240,1494519140,15,TESTS,OK,799,D,None
6,tourist,27021179,GNU C++11,9932800,CONTESTANT,4,2500,1719,1494518619,2000,PRETESTS,TIME_LIMIT_EXCEEDED,799,E,None
7,tourist,27019416,GNU C++11,28979200,CONTESTANT,39,1000,1145,1494518045,389,TESTS,OK,799,B,None
8,tourist,27018858,GNU C++11,17920000,CONTESTANT,79,1500,979,1494517879,62,TESTS,OK,799,C,None
9,tourist,27018375,GNU C++11,0,CONTESTANT,93,500,834,1494517734,15,TESTS,OK,799,A,None


In [57]:
query = """
SELECT * FROM submissions WHERE "participantType"='CONTESTANT'
"""
cur.execute(query)
#pd.read_sql_query(query, con)

In [54]:
cur.rowcount

5326298

# rename and re-type columns
postgreSQL changes all queries to lower case unless in double quotes
some columns were defined as strings but somehow became integers

In [94]:
query = """
ALTER TABLE submissions RENAME COLUMN "participantType" TO participanttype;
ALTER TABLE submissions RENAME COLUMN "passedTestCount" TO passedtestcount;
ALTER TABLE submissions RENAME COLUMN "relativeTimeSeconds" TO relativetimeseconds;
ALTER TABLE submissions RENAME COLUMN "startTimeSeconds" TO starttimeseconds;
ALTER TABLE submissions RENAME COLUMN "timeMilliseconds" TO timemilliseconds;
ALTER TABLE submissions RENAME COLUMN "contestID" TO contestid;
ALTER TABLE submissions RENAME COLUMN "problemID" TO problemid;
ALTER TABLE submissions RENAME COLUMN "problemRating" TO problemrating;
"""
cur.execute(query)
con.commit()

In [127]:
query = """
ALTER TABLE submissions DROP COLUMN IF EXISTS problemrating
"""
cur.execute(query)
con.commit()

In [97]:
query = """
ALTER TABLE problem_rating RENAME COLUMN "contestID" TO contestid;
ALTER TABLE problem_rating RENAME COLUMN "problemID" TO problemid;
ALTER TABLE problem_rating RENAME COLUMN "problemRating" TO problemrating;
"""
cur.execute(query)
con.commit()

In [36]:
query = """
ALTER TABLE problem_rating ALTER COLUMN contestid TYPE text;
ALTER TABLE problem_info ALTER COLUMN contestid TYPE text;
"""
cur.execute(query)
con.commit()


In [3]:
query = """
ALTER TABLE tags ALTER COLUMN contestid TYPE text;
ALTER TABLE tags ALTER COLUMN problemid TYPE text;
"""
cur.execute(query)
con.commit()


In [13]:
query = """
ALTER TABLE contests RENAME COLUMN "durationSeconds" TO durationseconds;
ALTER TABLE contests RENAME COLUMN "relativeTimeSeconds" TO relativetimeseconds;
ALTER TABLE contests RENAME COLUMN "startTimeSeconds" TO starttimeseconds;
ALTER TABLE hacks RENAME COLUMN "creationTimeSeconds" TO creationtimeseconds;
ALTER TABLE hacks RENAME COLUMN "contestID" TO contestid;
ALTER TABLE hacks RENAME COLUMN "problemID" TO problemid;
ALTER TABLE hacks RENAME COLUMN "problemName" TO problemname;
"""
cur.execute(query)
con.commit()

In [20]:
con.rollback()
query="ALTER TABLE contests RENAME COLUMN id TO contestid;"
cur.execute(query)
con.commit()

In [21]:
con.rollback()
query="ALTER TABLE hacks RENAME COLUMN id TO hackid;"
cur.execute(query)
con.commit()

In [22]:
con.rollback()
query="ALTER TABLE submissions RENAME COLUMN id TO submissionid;"
cur.execute(query)
con.commit()

In [109]:
con.rollback()

# Remove duplicate rows
Some tables (submissions) have duplicated rows, need to remove this to set a primary key

More on ctids here: https://www.postgresql.org/docs/8.2/static/ddl-system-columns.html

**ctid**
The physical location of the row version within its table. Note that although the ctid can be used to locate the row version very quickly, a row's ctid will change each time it is updated or moved by VACUUM FULL. Therefore ctid is useless as a long-term row identifier. The OID, or even better a user-defined serial number, should be used to identify logical rows.

In [12]:
con.rollback()
query = """
DELETE FROM submissions a USING (
      SELECT MIN(ctid) as ctid, submissionid
        FROM submissions 
        GROUP BY submissionid HAVING COUNT(*) > 1
      ) b
      WHERE a.submissionid = b.submissionid 
      AND a.ctid <> b.ctid
"""
cur.execute(query)
con.commit()

In [13]:
#	contestid	contestname	handle	newrating	oldrating	rank	ratingupdatetimeseconds
#0	785	Codeforces Round #404 (Div. 2)	tasneemria	1352	1327	1928	1489598100
#1	785	Codeforces Round #404 (Div. 2)	tasneemria	1352	1327	1928	1489598100
query = """
DELETE FROM user_rating a USING (
      SELECT MIN(ctid) as ctid, contestid, handle, ratingupdatetimeseconds
        FROM user_rating
        GROUP BY (contestid, handle, ratingupdatetimeseconds) HAVING COUNT(*) > 1
      ) b
      WHERE 
          a.contestid = b.contestid AND
          a.handle = b.handle AND
          a.ratingupdatetimeseconds = b.ratingupdatetimeseconds 
      AND a.ctid <> b.ctid
"""
cur.execute(query)
con.commit()

# Adding primary keys
https://www.postgresql.org/docs/9.2/static/ddl-constraints.html

This creates B-trees based on the primary keys

Examples:

http://www.postgresqltutorial.com/postgresql-foreign-key/

Tables:
* handles
* problem_info
* submissions
* contests
* hacks
* user_rating
* problem_rating
* tags

In [ ]:
con.rollback()
query = """
ALTER TABLE handles ADD PRIMARY KEY (handle);
"""
cur.execute(query)
con.commit()


In [16]:
query = """
ALTER TABLE contests ADD PRIMARY KEY (contestid);
ALTER TABLE hacks DROP CONSTRAINT hacks_pkey;
ALTER TABLE hacks ADD PRIMARY KEY (hackid, verdict, defender, hacker, contestid, problemid);
ALTER TABLE problem_info ADD PRIMARY KEY (contestid, problemid, division);
ALTER TABLE problem_rating ADD PRIMARY KEY (contestid, problemid);
ALTER TABLE submissions ADD PRIMARY KEY (handle, contestid, problemid, participanttype, verdict, submissionid, starttimeseconds);
ALTER TABLE tags ADD PRIMARY KEY (contestid, problemid, tag);
ALTER TABLE user_rating ADD PRIMARY KEY (contestid, handle);
"""
cur.execute(query)

ProgrammingError: multiple primary keys for table "contests" are not allowed


In [15]:
con.commit()

In [17]:
con.rollback()
query = """
SELECT * FROM user_rating  WHERE contestid=785 AND handle='tasneemria';
"""
pd.read_sql(query, con)

,contestid,contestname,handle,newrating,oldrating,rank,ratingupdatetimeseconds
0,785,Codeforces Round #404 (Div. 2),tasneemria,1352,1327,1928,1489598100


In [18]:
con.rollback()
query = """
SELECT * FROM submissions WHERE
    handle='zhanyl' AND
    contestid='100186' AND
    problemid='L' AND
    participanttype='VIRTUAL' AND
    verdict='OK' AND
    submissionid=7254239 AND
    starttimeseconds=1406433825
"""
cur.execute(query)
pd.read_sql_query(query, con)
#con.commit()


,handle,submissionid,language,memoryBytes,participanttype,passedtestcount,points,relativetimeseconds,starttimeseconds,timemilliseconds,testset,verdict,contestid,problemid
0,zhanyl,7254239,GNU C++,204800,VIRTUAL,20,0,225,1406433825,60,TESTS,OK,100186,L


# Exploratory analysis

In [33]:
query = """
SELECT * FROM submissions 
    INNER JOIN problem_rating 
        USING
        (
            problemid,
            contestid
        )
     WHERE handle='tourist'
"""
     #order by starttimeseconds, submissions.contestid, submissions.problemid
df_user = pd.read_sql_query(query, con)

In [34]:
query = """
SELECT * FROM user_rating
"""
df_rating = pd.read_sql_query(query, con)


In [35]:
df_rating.head()

,contestid,contestname,handle,newrating,oldrating,rank,ratingupdatetimeseconds
0,168,Codeforces Round #114 (Div. 2),Shoutmon,1209,1255,944,1332867600
1,205,Codeforces Round #129 (Div. 2),Shoutmon,1315,1209,475,1342027800
2,208,Codeforces Round #130 (Div. 2),Shoutmon,1257,1315,1091,1343065200
3,215,Codeforces Round #132 (Div. 2),Shoutmon,1404,1257,388,1344274200
4,216,Codeforces Round #133 (Div. 2),Shoutmon,1532,1404,241,1344965400


In [129]:
df_user

,problemid,contestid,handle,id,language,memoryBytes,participanttype,passedtestcount,points,relativetimeseconds,starttimeseconds,timemilliseconds,testset,verdict,problemrating
0,A,2,tourist,7267,Delphi,307200,CONTESTANT,9,0,473,1267117673,10,TESTS,WRONG_ANSWER,1291
1,A,2,tourist,7331,Delphi,307200,CONTESTANT,20,0,814,1267118014,30,TESTS,OK,1291
2,B,2,tourist,7421,Delphi,16691200,CONTESTANT,30,0,1406,1267118606,5000,TESTS,TIME_LIMIT_EXCEEDED,1621
3,B,2,tourist,7476,Delphi,16691200,CONTESTANT,34,0,1701,1267118901,380,TESTS,WRONG_ANSWER,1621
4,B,2,tourist,7531,Delphi,16691200,CONTESTANT,37,0,2168,1267119368,330,TESTS,OK,1621
5,C,2,tourist,7848,Delphi,409600,CONTESTANT,1,0,5218,1267122418,10,TESTS,WRONG_ANSWER,2019
6,C,2,tourist,7874,Delphi,409600,CONTESTANT,1,0,5436,1267122636,10,TESTS,WRONG_ANSWER,2019
7,C,2,tourist,7913,Delphi,409600,CONTESTANT,9,0,5811,1267123011,30,TESTS,RUNTIME_ERROR,2019
8,C,2,tourist,7962,Delphi,409600,CONTESTANT,9,0,6386,1267123586,10,TESTS,RUNTIME_ERROR,2019
9,C,2,tourist,7983,Delphi,409600,CONTESTANT,9,0,6524,1267123724,10,TESTS,RUNTIME_ERROR,2019


In [142]:
groups = df_user.groupby(['contestid', 'problemid'])
for k, v in groups:
    print k
    df = v
    break

('1', 'A')


# ideas
Get last 10 solved problems before each contest

**features**
- average time of solves before contest
- mean, var, max # wrong tries on problems
- mean, var of amt time between solves
- mean, var of difference between rating and problem rating
- total # problems solved
- \# problems solved where rating > x
- tags??
- contest vs practice questions


**Knowledge graph**

distribution of accuracy by tags (and problem rating?) by group of users of a certain rating

Can look at difference in this distribution between low and high users


**NLP**

Look at contest posts to see how much users liked a contest

predict problem tags from problem texts --> can apply to other websites
- trivial examples will include things like "tree" --> graph


** null model **
linear regression on difference, assume always the same

In [148]:
df = groups.get_group(('97', 'D'))
df.sort_values('starttimeseconds')

,problemid,contestid,handle,id,language,memoryBytes,participanttype,passedtestcount,points,relativetimeseconds,starttimeseconds,timemilliseconds,testset,verdict,problemrating
237,D,97,tourist,547491,Delphi,3174400,CONTESTANT,71,2000,7175,1310738375,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
238,D,97,tourist,547756,Delphi,3072000,PRACTICE,0,2000,20259,1310751459,10,TESTS,RUNTIME_ERROR,2682
239,D,97,tourist,547758,Delphi,6451200,PRACTICE,53,2000,20290,1310751490,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
240,D,97,tourist,547775,GNU C++,6348800,PRACTICE,53,2000,20578,1310751778,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
241,D,97,tourist,547776,Delphi,3174400,PRACTICE,71,2000,20605,1310751805,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
242,D,97,tourist,547778,Delphi,3174400,PRACTICE,71,2000,20639,1310751839,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
243,D,97,tourist,547780,Delphi,3174400,PRACTICE,70,2000,20666,1310751866,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
244,D,97,tourist,547781,Delphi,3174400,PRACTICE,70,2000,20690,1310751890,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
245,D,97,tourist,547783,Delphi,3174400,PRACTICE,71,2000,20710,1310751910,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
246,D,97,tourist,547787,Delphi,2867200,PRACTICE,70,2000,20795,1310751995,4000,TESTS,TIME_LIMIT_EXCEEDED,2682
